In [ ]:
import subprocess
import time
from pathlib import Path
import socket
from urllib.parse import urlparse
import shutil
import requests
import pandas as pd

print('All imports are ready in one place.')

In [ ]:
class OrchestrationCompose:
    def __init__(self, pattern='orchestration', mysql_service='mysql_orch'):
        self.pattern = pattern
        self.compose_dir = Path.cwd() / f"{self.pattern}_pattern"
        self.compose_cmd = self._detect_compose_cmd()
        self.mysql_service = mysql_service

    def _detect_compose_cmd(self):
        p = subprocess.run(['docker', 'compose', 'version'], capture_output=True)
        if p.returncode == 0:
            return ['docker', 'compose']
        p2 = subprocess.run(['docker-compose', '--version'], capture_output=True)
        if p2.returncode == 0:
            return ['docker-compose']
        raise RuntimeError('Neither `docker compose` nor `docker-compose` found')

    def check_compose_file(self):
        compose_path = self.compose_dir / 'compose.yaml'
        if not compose_path.exists():
            compose_path = self.compose_dir / 'docker-compose.yml'
        if not compose_path.exists():
            raise FileNotFoundError(f'No compose file found in {self.compose_dir}')

    def find_conflicts(self):
        expected_conflicts = [
            'cloudmart-mysql-orchestration',
            'cloudmart-redis-orchestration',
            'cloudmart-rabbitmq-orchestration',
            'orchestration-order-service',
            'orchestration-inventory-service',
            'orchestration-payment-service',
            'orchestration-shipping-service'
        ]
        conflicts = []
        for name in expected_conflicts:
            p = subprocess.run(['docker', 'ps', '-a', '--filter', f'name={name}', '--format', '{{.Names}}'], capture_output=True, text=True)
            if p.stdout.strip():
                conflicts.append(p.stdout.strip())
        return conflicts

    def up(self, timeout=600):
        cmd = ' '.join(self.compose_cmd) + ' up -d --build'
        print('RUN:', cmd)
        p = subprocess.run(cmd, cwd=str(self.compose_dir), shell=True, capture_output=True, text=True, timeout=timeout)
        print('returncode', p.returncode)
        if p.stdout:
            print('STDOUT:', p.stdout[:2000])
        if p.stderr:
            print('STDERR:', p.stderr[:2000])
        return p

    def exec_in_mysql(self, cmd, timeout=60):
        composed = ' '.join(self.compose_cmd) + f" exec -T {self.mysql_service} {cmd}"
        p = subprocess.run(composed, cwd=str(self.compose_dir), shell=True, capture_output=True, text=True, timeout=timeout)
        return p

def is_port_open(host: str, port: int, timeout: float = 1.0) -> bool:
    """Check if a TCP port is open on the given host."""
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except Exception:
        return False

def wait_for_port(url: str, timeout: int = 60, interval: float = 1.0) -> bool:
    """Wait until the host:port in the URL accepts TCP connections."""
    parsed = urlparse(url)
    host = parsed.hostname or 'localhost'
    port = parsed.port or (443 if parsed.scheme == 'https' else 80)
    start = time.time()
    while time.time() - start < timeout:
        if is_port_open(host, port, timeout=1.0):
            return True
        time.sleep(interval)
    return False

print('Helper classes and functions are defined.')

# Step 1: Pre-check and Environment Setup

This cell checks for Docker, determines the correct `docker-compose` command, and verifies that no conflicting containers are running. This is a safety check to prevent errors when starting the services.

In [ ]:
pattern = 'orchestration'
compose_dir = Path.cwd() / f"{pattern}_pattern"
print('Compose dir:', compose_dir)

# Check docker CLI
if shutil.which('docker') is None:
    print('ERROR: `docker` not found in PATH. Please install Docker Desktop or ensure `docker` is available.')
    raise SystemExit(1)

# Determine compose command: prefer `docker compose` (v2) then `docker-compose` (v1)
compose_cmd = None
p = subprocess.run(['docker', 'compose', 'version'], capture_output=True, text=True)
if p.returncode == 0:
    compose_cmd = ['docker', 'compose']
else:
    p2 = subprocess.run(['docker-compose', '--version'], capture_output=True, text=True)
    if p2.returncode == 0:
        compose_cmd = ['docker-compose']

if compose_cmd is None:
    print('ERROR: Neither `docker compose` nor `docker-compose` command is available.')
    raise SystemExit(1)

print('Using compose command:', ' '.join(compose_cmd))

# Check compose file exists
if not (compose_dir / 'compose.yaml').exists() and not (compose_dir / 'docker-compose.yml').exists():
    print(f'ERROR: No compose file found in {compose_dir} (expected compose.yaml or docker-compose.yml).')
    raise SystemExit(1)

# Check for existing conflicting containers (do not auto-remove)
expected_conflicts = [
    'cloudmart-mysql-orchestration',
    'cloudmart-redis-orchestration',
    'cloudmart-rabbitmq-orchestration',
    'orchestration-order-service',
    'orchestration-inventory-service',
    'orchestration-payment-service',
    'orchestration-shipping-service'
    ]
conflicts = []
for name in expected_conflicts:
    p = subprocess.run(['docker', 'ps', '-a', '--filter', f'name={name}', '--format', '{{.Names}}'], capture_output=True, text=True)
    if p.stdout.strip():
        conflicts.append(p.stdout.strip())

if conflicts:
    print('Found existing containers that may conflict:', conflicts)
    print('Recommended actions:')
    print('  1) Manually stop and remove these containers:')
    print('       docker rm -f ' + ' '.join(conflicts))
    print('  2) Or run the compose directory command to tear down the project:')
    print(f'       cd {compose_dir} && {" ".join(compose_cmd)} down --remove-orphans')
    # --- Run the recommended command automatically ---
    import os
    print('Running: cd /Users/codefox/workspace/practice_infra_arch/saga_pattern/orchestration_pattern && docker compose down --remove-orphans')
    os.chdir('/Users/codefox/workspace/practice_infra_arch/saga_pattern/orchestration_pattern')
    subprocess.run(['docker', 'compose', 'down', '--remove-orphans'])
    print('Conflicting containers removed. Please rerun this cell if needed.')
    raise SystemExit(0)

print('No conflicting containers found. Safe to run DB init cell.')


Compose dir: /Users/codefox/workspace/practice_infra_arch/saga_pattern/orchestration_pattern
Using compose command: docker compose
No conflicting containers found. Safe to run DB init cell.


In [ ]:
def is_port_open(host: str, port: int, timeout: float = 1.0) -> bool:
    """Check if a TCP port is open on the given host."""
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except Exception:
        return False

def wait_for_port(url: str, timeout: int = 60, interval: float = 1.0) -> bool:
    """Wait until the host:port in the URL accepts TCP connections."""
    parsed = urlparse(url)
    host = parsed.hostname or 'localhost'
    port = parsed.port or (443 if parsed.scheme == 'https' else 80)
    start = time.time()
    while time.time() - start < timeout:
        if is_port_open(host, port, timeout=1.0):
            return True
        time.sleep(interval)
    return False

# Step 2: Start Services and Initialize Database

This cell starts the Docker containers for the orchestration pattern and waits for the MySQL database to become healthy. It then runs an initialization script to set up the database schema and initial data.

In [ ]:
compose_dir = Path.cwd() / "orchestration_pattern"
print(f"Ensuring services are up: cd {compose_dir} && docker compose up -d --build")
subprocess.run(["docker", "compose", "up", "-d", "--build"], cwd=str(compose_dir))

out_csv = Path.cwd().parent / 'test_results.csv'  # saga_pattern/test_results.csv

# Service endpoints (matching compose mappings)
CHOREO_ORDER_URL = 'http://localhost:8001/orders'      # choreography order-service
ORCH_ORCHESTRATOR_URL = 'http://localhost:8005/orders' # orchestrator entry
ORCH_ORDER_HOST_PORT = 'http://localhost:8011/orders'  # orchestration's order-service mapped to 8011

# Define scenarios
scenarios = [
    ("choreography", "success", CHOREO_ORDER_URL, {"customerId":"customer-001","bookId":"book-123","quantity":1}),
    ("choreography", "stock_failure", CHOREO_ORDER_URL, {"customerId":"customer-002","bookId":"book-456","quantity":1}),
    ("choreography", "payment_failure", CHOREO_ORDER_URL, {"customerId":"customer-004","bookId":"book-789","quantity":1}),
    ("orchestration", "orchestrator_submit", ORCH_ORCHESTRATOR_URL, {"customerId":"customer-001","bookId":"book-123","quantity":1}),
    ("orchestration", "order_service_direct", ORCH_ORDER_HOST_PORT, {"customerId":"customer-001","bookId":"book-123","quantity":1}),
]

results = []

# Run scenarios with port wait and unreachable handling
for pattern, name, url, payload in scenarios:
    parsed = urlparse(url)
    host = parsed.hostname or 'localhost'
    port = parsed.port or (443 if parsed.scheme == 'https' else 80)

    # Measure wait duration separately
    wait_start = time.time()
    reachable = wait_for_port(url, timeout=60, interval=1.0)

    if not reachable:
        elapsed = time.time() - wait_start
        results.append({
            'pattern': pattern,
            'scenario': name,
            'url': url,
            'status_code': 'UNREACHABLE',
            'response_time': elapsed,
            'result': f'Host {host}:{port} not reachable within timeout'
        })
        continue

    # If reachable, measure HTTP request latency only
    req_start = time.time()
    try:
        r = requests.post(url, json=payload, timeout=10)
        elapsed = time.time() - req_start
        try:
            body = r.json()
        except Exception:
            body = r.text
        status_code = r.status_code
    except requests.exceptions.RequestException as e:
        elapsed = time.time() - req_start
        status_code = getattr(getattr(e, 'response', None), 'status_code', 'N/A')
        body = str(e)
    results.append({
        'pattern': pattern,
        'scenario': name,
        'url': url,
        'status_code': status_code,
        'response_time': elapsed,
        'result': body
    })

# Build DataFrame and save CSV
expected_cols = ['pattern', 'scenario', 'url', 'status_code', 'response_time', 'result']
df = pd.DataFrame(results, columns=expected_cols)
df['result'] = df['result'].apply(lambda x: str(x) if x is not None else '')
out_csv.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(out_csv, index=False)
print('Wrote test results to', out_csv)
print(df)

Ensuring services are up: cd /Users/codefox/workspace/practice_infra_arch/saga_pattern/orchestration_pattern && docker compose up -d --build
#1 [internal] load local bake definitions
#1 reading from stdin 3.01kB done
#1 DONE 0.0s
#1 [internal] load local bake definitions
#1 reading from stdin 3.01kB done
#1 DONE 0.0s

#2 [payment-service internal] load build definition from Dockerfile
#2 DONE 0.0s

#3 [shipping-service internal] load build definition from Dockerfile
#3 transferring dockerfile: 664B done
#3 DONE 0.0s

#4 [order-service internal] load build definition from Dockerfile
#4 transferring dockerfile: 658B done
#4 DONE 0.0s

#2 [payment-service internal] load build definition from Dockerfile
#2 transferring dockerfile: 662B done
#2 DONE 0.0s

#5 [saga-orchestrator internal] load build definition from Dockerfile
#5 transferring dockerfile: 666B done
#5 DONE 0.0s

#6 [inventory-service internal] load build definition from Dockerfile
#6 transferring dockerfile: 666B done
#6 DONE 0

 orchestration_pattern-inventory-service  Built
 orchestration_pattern-payment-service  Built
 orchestration_pattern-shipping-service  Built
 orchestration_pattern-saga-orchestrator  Built
 orchestration_pattern-order-service  Built
 Container cloudmart-rabbitmq-orchestration  Running
 Container cloudmart-mysql-orchestration  Running
 Container cloudmart-redis-orchestration  Running
 Container cloudmart-mysql-orchestration  Waiting
 Container cloudmart-redis-orchestration  Waiting
 Container cloudmart-redis-orchestration  Waiting
 Container cloudmart-mysql-orchestration  Waiting
 Container cloudmart-mysql-orchestration  Waiting
 Container cloudmart-redis-orchestration  Waiting
 Container cloudmart-redis-orchestration  Waiting
 Container cloudmart-mysql-orchestration  Waiting
 Container cloudmart-mysql-orchestration  Waiting
 Container cloudmart-redis-orchestration  Waiting
 Container cloudmart-rabbitmq-orchestration  Waiting
 Container cloudmart-redis-orchestration  Healthy
 Container 

Wrote test results to /Users/codefox/workspace/practice_infra_arch/test_results.csv
         pattern              scenario                           url  \
0   choreography               success  http://localhost:8001/orders   
1   choreography         stock_failure  http://localhost:8001/orders   
2   choreography       payment_failure  http://localhost:8001/orders   
3  orchestration   orchestrator_submit  http://localhost:8005/orders   
4  orchestration  order_service_direct  http://localhost:8011/orders   

   status_code  response_time  \
0  UNREACHABLE      60.256015   
1  UNREACHABLE      60.263873   
2  UNREACHABLE      60.264533   
3          404       0.019062   
4          500       0.017411   

                                             result  
0  Host localhost:8001 not reachable within timeout  
1  Host localhost:8001 not reachable within timeout  
2  Host localhost:8001 not reachable within timeout  
3                           {'detail': 'Not Found'}  
4             

In [ ]:
oc = OrchestrationCompose(pattern='orchestration')
oc.check_compose_file()
conflicts = oc.find_conflicts()
if conflicts:
    print('Conflicts found:', conflicts)
    raise SystemExit(1)

res = oc.up()
print('Compose up finished with return code', res.returncode)

# Wait for MySQL healthy
mysql_ok = False
for i in range(60):
    p = oc.exec_in_mysql('mysqladmin ping -h localhost -ucloudmart_user -pcloudmart_pass', timeout=20)
    if p.returncode == 0 and 'mysqld is alive' in (p.stdout + p.stderr):
        mysql_ok = True
        print('MySQL reported healthy')
        break
    time.sleep(2)

if not mysql_ok:
    print('Warning: MySQL did not enter healthy state within timeout')

# Run init script if present
init_path = '/docker-entrypoint-initdb.d/init.sql'
init_check = oc.exec_in_mysql(f'sh -c "test -f {init_path} && echo exists"')
if 'exists' in (init_check.stdout + init_check.stderr):
    cmd = f'sh -c "mysql -ucloudmart_user -pcloudmart_pass cloudmart_saga < {init_path}"'
    r = oc.exec_in_mysql(cmd)
    print('Init script returncode', r.returncode)
else:
    print('No init script found inside container; skipping')


# Step 3: Run Smoke Tests

This cell runs a series of smoke tests against both the choreography and orchestration patterns. It checks for service availability and then sends test requests to simulate different scenarios (success, stock failure, payment failure). The results are collected and saved to a CSV file for analysis.